In [1]:
%pip install BeautifulSoup4 -U

Requirement already up-to-date: BeautifulSoup4 in /home/abel/.local/lib/python3.8/site-packages (4.11.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import random
import time
import re

# scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from keys import EXTENSION_LINK, API_KEY

In [3]:
headers = {
     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.32  (KHTML, like Gecko) Chrome/75.0.3770.139 Safari/537.32'
}

In [4]:
url_partidos = "https://www.tse.jus.br/partidos/partidos-registrados-no-tse"
request_partidos = requests.get(url_partidos, headers=headers)
request_partidos.encoding = 'utf-8'
print(f"Status code {url_partidos}: {request_partidos.status_code}")

Status code https://www.tse.jus.br/partidos/partidos-registrados-no-tse: 200


In [5]:
# Pega todos os partidos existentes
soup_partidos = BeautifulSoup(request_partidos.text, 'html.parser')
lista_nome_partidos = soup_partidos \
    .find('table', class_='grid listing table') \
    .find('tbody') \
    .find_all('tr')

# Filtragem e relação dos partidos com data de registro
lista_nome_partidos.pop(0)
lista_nome_partidos.pop(-1)

partido_ano = {}
for linha in lista_nome_partidos:
    element = linha.find_all('td')
    partido = element[1].find('strong').text
    ano = element[3].text
    partido_ano[partido] = ano

# Adiciona todos os nomes polssíveis para o partido
if partido_ano["MDB"]:
    partido_ano[("PMDB", "MDB")] = partido_ano.pop("MDB")

print(partido_ano)

{'PTB': '3.11.1981', 'PDT': '10.11.1981', 'PT': '11.2.1982', 'PCdoB': '23.6.1988', 'PSB': '1°.7.1988', 'PSDB': '24.8.1989', 'AGIR': '22.2.1990', 'PSC': '29.3.1990', 'PMN': '25.10.1990', 'CIDADANIA': '19.3.1992', 'PV': '30.9.1993', 'AVANTE': '11.10.1994', 'PP': '16.11.1995', 'PSTU': '19.12.1995', 'PCB': '9.5.1996', 'PRTB': '18.2.1997', 'DC': '5.8.1997', 'PCO': '30.9.1997', 'PODE': '2.10.1997', 'REPUBLICANOS': '25.8.2005', 'PSOL': '15.9.2005', 'PL': '19.12.2006', 'PSD': '27.9.2011', 'PATRIOTA': '19.6.2012', 'PROS': '24.9.2013', 'SOLIDARIEDADE': '24.9.2013', 'NOVO': '15.9.2015', 'REDE': '22.9.2015', 'PMB': '29.9.2015', 'UP': '10.12.2019', 'UNIÃO': '8.2.2022', ('PMDB', 'MDB'): '30.6.1981'}


In [6]:
# Segunda Consulta
url_prestacao = "https://www.tse.jus.br/partidos/contas-partidarias/prestacao-de-contas/contas-anuais/prestacao-de-contas-partidarias"
request_prestacao = requests.get(url_prestacao, headers=headers)
request_prestacao.encoding = 'utf-8'

print(f"Status code {url_prestacao}: {request_prestacao.status_code}")

Status code https://www.tse.jus.br/partidos/contas-partidarias/prestacao-de-contas/contas-anuais/prestacao-de-contas-partidarias: 200


In [8]:
# Pega o link de 2020
soup_anos = BeautifulSoup(request_prestacao.text, 'html.parser')
lista_anos = soup_anos.find('div', class_= "tab-pane active").find_all('a')
for i in lista_anos:
    if "2020" in i.text:
        link_2020 = i.get('href')
        break
    else:
        print('Não encontrou o link de 2020')

In [14]:
# Função parar tempos aleatórios
def random_time(x, y):
    time.sleep(random.uniform(x, y))

# Configura o Driver
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_extension('Buster.crx.crx')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# Configuração da Extensão
driver.get(EXTENSION_LINK)
wait = WebDriverWait(driver, 30)
wait.until(EC.element_to_be_clickable((By.CLASS_NAME ,"mdc-select"))).click()
wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='mdc-list']/li[5]"))).click()
wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='mdc-text-field mdc-ripple-upgraded']/input"))).send_keys(API_KEY)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/home/abel/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_198137/1586555328.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [15]:
request_ano = requests.get(link_2020, headers=headers)
request_ano.encoding = 'utf-8'
print(f"Status code {link_2020}: {request_ano.status_code}")

soup_demost = BeautifulSoup(request_ano.text, 'html.parser')
lista_processos = soup_demost \
    .find('table', class_= "plain table") \
    .find('tbody') \
    .find_all('tr')
lista_processos.pop(0)

Status code https://www.tse.jus.br/partidos/contas-partidarias/prestacao-de-contas/integra-da-prestacao-de-contas-e-demonstrativos-dos-partidos-referentes-a-2020: 200


<tr>
<td style="text-align: center; "><strong>SIGLA</strong></td>
<td style="text-align: center; "><strong>PARTIDO</strong></td>
<td style="text-align: center; "><strong>N°PROCESSO (PJe)</strong></td>
</tr>

In [16]:
partido_processo = {}
for linha in lista_processos[:1]: # Talvez temporário
    element = linha.find_all('td')
    partido = element[0].text
    processo = element[2].text.strip('\n')
    partido_processo[partido] = processo

In [18]:
# Faz nova consulta para cada partido usando Selenium
for partido, processo in partido_processo.items():
    url_processo = f"https://consultaunificadapje.tse.jus.br/#/public/resultado/{processo}"

    # Resolve o problema do ReCAPTCHA
    driver.get(url_processo)
    
    driver.switch_to.frame(driver.find_elements(by=By.TAG_NAME, value="iframe")[0])
    checkbox = wait.until(EC.presence_of_element_located((By.ID ,"recaptcha-anchor")))
    random_time(0.5, 1.5)
    checkbox.click()
    
    random_time(2, 3)
    driver.switch_to.default_content()
    try:
        driver.switch_to.frame(driver.find_elements(by=By.TAG_NAME, value="iframe")[2])
        reacptcha_sound = wait.until(EC.presence_of_element_located((By.XPATH , '//*[@class="rc-buttons"]/div[2]')))
        random_time(0.5, 1.5)
        reacptcha_sound.click()

        random_time(2, 3)
        driver.switch_to.default_content()
        driver.switch_to.frame(driver.find_elements(by=By.TAG_NAME, value="iframe")[2])
        reacptcha_sound_resolver = wait.until(EC.presence_of_element_located((By.XPATH , '//*[@class="rc-buttons"]/div[4]')))
        random_time(0.5, 1.5)
        reacptcha_sound_resolver.click()

    except:
        pass